In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

# Get data from our "database"

In [ ]:
# See: https://towardsdatascience.com/read-data-from-google-sheets-into-pandas-without-the-google-sheets-api-5c468536550
sheet_id = "1q8MB-H49pd1ojqZzMbs0ExbMkWKl0Ll0cyJ3_OpF3_g"
sheet_name = "Job_Search_Status"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url, usecols=range(0, 11))

# Data Analysis

## Where are we finding jobs?

In [ ]:
nycdsa = (
    df.groupby("Source")["Company"]
    .nunique()
    .reset_index()
    .rename(columns={"Company": "Positions"})
)
fig = px.pie(nycdsa, values="Positions", names="Source", title="Applications by Source")
colors = ["gold", "mediumturquoise", "darkorange", "lightgreen"]
fig.update_traces(
    hoverinfo="label+percent",
    textinfo="value",
    textfont_size=20,
    marker=dict(colors=colors, line=dict(color="#151515", width=2)),
)
fig.show()

## What industries are we targeting?
Note: the data is not complete. We don't know the industry for each job

In [ ]:
industry = (
    df.groupby("Industry")["Company"]
    .nunique()
    .reset_index()
    .rename(columns={"Company": "Jobs"})
    .sort_values(by="Jobs", ascending=False)
    .head(10)
)

fig = px.bar(industry, title="Jobs by Industry (top 10)", x="Industry", y="Jobs")
fig.show()

## Let's see our effort over time

In [ ]:
apps_time = (
    df.groupby("App Date")[["Company"]]
    .nunique()
    .reset_index()
    .rename(columns={"Company": "Applications", "App Date": "Date"})
)
apps_time["Date"] = pd.to_datetime(apps_time["Date"])
apps_time["Applications"] = apps_time["Applications"].cumsum()

fig = px.line(
    apps_time, x="Date", y="Applications", title="Number of applications over time"
)
fig.show()

## What is the current status across our applications?

In [ ]:
today = datetime.today()

# Split status `Applied` -> `Open` and `No Response` categories
app_status = df[["Status", "App Date"]]
app_status["App Date"] = pd.to_datetime(app_status["App Date"])
app_status["Status"] = app_status.apply(
    lambda row: "Open"
    if (row["Status"] == "Applied" and (today - row["App Date"]).days < 10)
    else "No response"
    if row["Status"] == "Applied"
    else row["Status"],
    axis=1,
)
app_status = app_status.Status.value_counts().reindex(
    index=[
        "Open",
        "No response",
        "Rejected",
        "Interview",
        "Onsite",
        "Chosen",
        "Accepted",
    ]
)

In [ ]:
fig = px.bar(
    app_status,
    color=app_status.index,
    labels={"value": "Applications", "index": "Status"},
    title=f"Application status ({app_status.sum()} positions)",
)
fig.show()

## Let's see this whole picture!

In [ ]:
# add application status. NOTE: we update the `Interview` value /after/ the total
total_applications = app_status.sum()
app_status["Interview"] = (
    app_status.Interview + app_status.Onsite + app_status.Chosen + app_status.Accepted
)

status_values = [total_applications] + list(app_status.values)
status_labels = ["Applications"] + list(app_status.index)

status_colors = [
    "chartreuse",
    "rgba(158,202,225,1)",
    "rgba(254,178,76,1)",
    "rgba(240,59,32,1)",
    "rgba(247,252,185,1)",
    "rgba(173,221,142,1)",
    "rgba(49,163,84,1)",
    "rgba(44,127,184,1)",
]
status_link_colors = [
    "rgba(158,202,225,.4)",
    "rgba(254,178,76,.2)",
    "rgba(240,59,32,.2)",
    "rgba(247,252,185,.5)",
    "rgba(173,221,142,.2)",
    "rgba(49,163,84,.2)",
    "rgba(44,127,184,.2)",
]

# also add application source as input into start node
source_values = list(df.Source.value_counts().values)
source_labels = list(df.Source.value_counts().index)
source_colors = [
    "rgba(239,237,245,1)",
    "rgba(239,237,245,1)",
    "rgba(188,189,220,1)",
    "rgba(117,107,177,1)",
]  # FAKE is duplicated color here
source_link_colors = [
    "rgba(239,237,245,.5)",
    "rgba(239,237,245,.5)",
    "rgba(188,189,220,.5)",
    "rgba(117,107,177,.5)",
]

# merge job source and job status
values = source_values + status_values
labels = source_labels + status_labels
labels_with_values = ["%s: %s" % x for x in zip(labels, values)]
colors = source_colors + status_colors
link_colors = source_link_colors + status_link_colors

source_sources = [0, 1, 2, 3]
source_targets = [4, 4, 4, 4]
status_sources = [x + len(source_sources) for x in [0, 0, 0, 0, 4, 5, 6, 7]]
status_targets = [x + len(source_targets) for x in [1, 2, 3, 4, 5, 6, 7, 8]]
sources = source_sources + status_sources
targets = source_targets + status_targets

# HACK remove the value for "Application"
del values[len(source_sources)]

fig = go.Figure(
    data=[
        go.Sankey(
            arrangement="freeform",
            valueformat=".0f",
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="#151515", width=0.5),
                label=labels_with_values,
                color=colors,
                x=[0.1, 0.1, 0.1, 0.1, 0.25, 0.35, 0.4, 0.4, 0.45, 0.5, 0.55, 0.65],
                y=[0.2, 0.7, 0.8, 1.0, 0.50, 0.10, 0.5, 0.7, 1.00, 0.7, 0.60, 0.50],
                customdata=labels,
                hovertemplate="%{customdata}",
            ),
            link=dict(
                source=sources,
                target=targets,
                value=values,
                color=link_colors,
                hoverinfo="skip",
            ),
        )
    ]
)

fig.update_layout(
    title_text=f"Rishi's Job Search Status<br>{today.strftime('%Y-%m-%d')}<br>(Includes fake data)",
    font_size=10,
)
fig.show()